# Title
[]()

In [52]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [2]:
from silvhua_plot import plot_int_bar, plot_proportion

In [53]:
from Sheets_api import *

In [3]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 10)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Load previously saved dataframe

## References

In [4]:
pickle_path = r'C:/Users/silvh/OneDrive/lighthouse/Ginkgo coding/content-summarization/sources'
filename = 'ranked_kl_references_2023-04-28_1550.sav'
references = loadpickle(filename, pickle_path)
references

Time completed: 2023-05-03 10:41:39.900073
Dataframe shape:  (1514, 5)
DataFrame columns: ['Knowledge ID #', 'Citation #', 'Reference', 'Reference Type', 'Reference Rank']


,Knowledge ID #,Citation #,Reference,Reference Type,Reference Rank
0,2,1,Tseret...,Journa...,1
1,6,1,Paravl...,Journa...,1
2,8,1,"Jäger,...",Journa...,1
3,10,1,Pontze...,Journa...,1
4,11,1,Sinaki...,Journa...,1
...,...,...,...,...,...
1509,1209,3,What a...,Websit...,4
1510,1210,1,When s...,Websit...,4
1511,1210,2,Americ...,Websit...,4
1512,1211,1,How a ...,Websit...,4


## Data

In [5]:
filename = 'prompt chain results Google Sheet processed_2023-04-28_1350.sav'
pickle_path = r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
data = loadpickle(filename, pickle_path)
data.head()

Time completed: 2023-05-03 10:41:40.094211
Dataframe shape:  (129, 38)
DataFrame columns: ['A: date added', 'B: article', 'C: choice # for given prompt', 'D: system_role', 'E: model', 'F: text', 'G: prep_step', 'H: summarization task', 'I: full summarization task', 'J: original summary', 'K: original summary content rating', 'L: original summary language rating', 'M: top summary', 'N: simple summary choice', 'O: simplify audience', 'P: simplify task', 'Q: full simplify task', 'R: simple summary', 'S: simple summary content rating', 'T: simple summary language rating', 'U: top simple summary', 'V: add relevance task', 'W: full add relevance task: athletes', 'X: added relevance: athletes', 'Y: added relevance content rating', 'Z: added relevance language rating', 'AA: top added relevance statement', 'AB: add relevance task', 'AC: full add relevance task: older adults', 'AD: added relevance: older adults', 'AE: added relevance content rating', 'AF: added relevance language rating', 'AG: t

,A: date added,B: article,C: choice # for given prompt,D: system_role,E: model,F: text,G: prep_step,H: summarization task,I: full summarization task,J: original summary,K: original summary content rating,L: original summary language rating,M: top summary,N: simple summary choice,O: simplify audience,P: simplify task,Q: full simplify task,R: simple summary,S: simple summary content rating,T: simple summary language rating,U: top simple summary,V: add relevance task,W: full add relevance task: athletes,X: added relevance: athletes,Y: added relevance content rating,Z: added relevance language rating,AA: top added relevance statement,AB: add relevance task,AC: full add relevance task: older adults,AD: added relevance: older adults,AE: added relevance content rating,AF: added relevance language rating,AG: top added relevance statement,AH: Column 33,K: original summary ratings meet standard,S: simple summary ratings meet standard,Y: added relevance ratings meet standard,AE: added relevance ratings meet standard
0,2023-0...,Daily ...,1,You ar...,gpt-3....,"""Daily...",Take t...,Summar...,Take t...,"""New r...",1.0,4.0,NaN,,,,Use te...,Scient...,1.0,1.0,NaN,Add 1-...,Add 1-...,,NaN,NaN,NaN,Add 1-...,Add 1-...,None,NaN,NaN,NaN,None,0,0,0,0
1,2023-0...,Daily ...,2,You ar...,gpt-3....,"""Daily...",Take t...,Summar...,Take t...,A new ...,1.0,4.0,NaN,,,,Use te...,Scient...,1.0,1.0,NaN,Add 1-...,Add 1-...,,NaN,NaN,NaN,Add 1-...,Add 1-...,None,NaN,NaN,NaN,None,0,0,0,0
2,2023-0...,Daily ...,3,You ar...,gpt-3....,"""Daily...",Take t...,Summar...,Take t...,New re...,3.0,4.0,NaN,,,,Use te...,Scient...,3.0,1.0,NaN,Add 1-...,Add 1-...,,NaN,NaN,NaN,Add 1-...,Add 1-...,None,NaN,NaN,NaN,None,0,0,0,0
3,2023-0...,Daily ...,4,You ar...,gpt-3....,"""Daily...",Take t...,Summar...,Take t...,This s...,3.0,4.0,1.0,,,,Use te...,Scient...,3.0,1.0,1.0,Add 1-...,Add 1-...,For at...,4.0,4.0,1.0,Add 1-...,Add 1-...,These ...,4.0,4.0,1.0,None,0,0,1,1
4,2023-0...,Daily ...,5,You ar...,gpt-3....,"""Daily...",Take t...,Summar...,Take t...,New re...,1.0,4.0,NaN,,,,Use te...,Scient...,1.0,1.0,NaN,Add 1-...,Add 1-...,,NaN,NaN,NaN,Add 1-...,Add 1-...,None,NaN,NaN,NaN,None,0,0,0,0


# Pull updated ratings

# Functions from previous notebooks

## Update `process_sheet`: Meeting threshold scores

In [6]:
import re
import string
def evaluate_standard(df, column_groups, content_standard, language_standard):
    for standard_column, content_rating, language_rating  in column_groups:
        if (df[content_rating] >= content_standard) & (df[language_rating] >= language_standard):
            df[standard_column] = 1
        else:
            df[standard_column] = 0
    return df

def process_sheet(df, content_standard=4, language_standard=4, column_pairs=None):

    spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
    processed_df = df.copy()
    processed_df.columns = [f'{spreadsheet_columns[index]}: {column}' for index, column in enumerate(processed_df.columns)]

    # Extract the numbers from ratings
    ratings_columns = [column for column in processed_df.columns if ('rating' in column) | ('top' in column)]
    print(f'Ratings columns (n={len(ratings_columns)}):', ratings_columns)
    processed_df[ratings_columns] = processed_df[ratings_columns].apply(
        lambda x: x.str.extract(r'(\d+)', expand=False))
    processed_df[ratings_columns] = processed_df[ratings_columns].apply(pd.to_numeric, errors='coerce')
    # processed_df[ratings_columns] = processed_df[ratings_columns].astype('Int64')
    print(f'Ratings columns (n={len(processed_df[ratings_columns].columns)}): {[column for column in processed_df[ratings_columns].columns]}')

    # Strip white spaces at start and end of strings
    str_cols = processed_df.select_dtypes(include=['object']).columns
    print(f'String columns (n={len(str_cols)}): {[column for column in str_cols]}')
    processed_df[str_cols] = processed_df[str_cols].apply(lambda x: x.str.strip())
    print('White spaces stripped at start and end of strings')


    if column_pairs == None:
        content_ratings_columns = processed_df.columns[processed_df.columns.str.contains('content rating')]
        language_ratings_columns = processed_df.columns[processed_df.columns.str.contains('language rating')]
    new_column_names = [
        re.sub(r'content rating', 'ratings meet standard', column) for column in content_ratings_columns]
    column_groups = [(
        name, content_rating, language_rating
        ) for name, content_rating, language_rating in (zip(
        new_column_names, content_ratings_columns, language_ratings_columns))
        ]
    processed_df = processed_df.apply(
        evaluate_standard, args=(column_groups,  content_standard, language_standard), axis=1)
    
    return processed_df


# df = process_sheet(data)
# df['Q: full simplify task'].replace('','Use terms a 12-year-old can understand.', inplace=True)



# EDA

In [7]:
df = data
top_results_columns = df.columns[df.columns.str.contains('top')]
content_ratings_columns = df.columns[df.columns.str.contains('content rating')]
language_ratings_columns = df.columns[df.columns.str.contains('language rating')]
standards_columns = df.columns[df.columns.str.contains('meet standard')]
task_columns = ['H: summarization task', 'Q: full simplify task', 'V: add relevance task']
print(f'Top results columns (n={len(top_results_columns)}):\n\t', [column for column in top_results_columns])
print(f'Content ratings columns (n={len(content_ratings_columns)}):\n\t', [column for column in content_ratings_columns])
print(f'Language ratings columns (n={len(language_ratings_columns)})\n\t:', [column for column in language_ratings_columns])
print(f'Standards columns (n={len(standards_columns)}):\n\t', [column for column in standards_columns])
print(f'Task columns (n={len(task_columns)}):\n\t', [column for column in task_columns])

Top results columns (n=4):
	 ['M: top summary', 'U: top simple summary', 'AA: top added relevance statement', 'AG: top added relevance statement']
Content ratings columns (n=4):
	 ['K: original summary content rating', 'S: simple summary content rating', 'Y: added relevance content rating', 'AE: added relevance content rating']
Language ratings columns (n=4)
	: ['L: original summary language rating', 'T: simple summary language rating', 'Z: added relevance language rating', 'AF: added relevance language rating']
Standards columns (n=4):
	 ['K: original summary ratings meet standard', 'S: simple summary ratings meet standard', 'Y: added relevance ratings meet standard', 'AE: added relevance ratings meet standard']
Task columns (n=3):
	 ['H: summarization task', 'Q: full simplify task', 'V: add relevance task']


## plot abs_and_proportion 

### Which prompts result in good results?

In [42]:
def plot_proportion_abs(df, columns, classification, label=1, height=150):
    plot_proportion(df, columns, classification, label, height=height)
    plot_int_bar(df, columns, classification, label, height=height)

columns=[task_columns[0]]
classification=standards_columns[0]

plot_proportion_abs(df, columns, classification, label=1)

In [43]:
index = 1
columns=[task_columns[index]]
classification=standards_columns[index]

plot_proportion_abs(df, columns, classification, label=1)

In [17]:
index = 2
columns=[task_columns[index]]
classification=standards_columns[index]

plot_proportion_abs(df, columns, classification, label=1)

In [16]:
index = -1
columns=[task_columns[index]]
classification=standards_columns[index]

plot_proportion_abs(df, columns, classification, label=1)

### Does choice number affect quality?

Meets standards yes/no

In [36]:
choice_number_columns = df.columns[df.columns.str.contains('choice')].tolist()
print(f'Choice number columns (n={len(choice_number_columns)}):\n\t', [column for column in choice_number_columns])
print(f'Standard columns (n={len(standards_columns)}):\n\t', [column for column in standards_columns])

Choice number columns (n=2):
	 ['C: choice # for given prompt', 'N: simple summary choice']
Standard columns (n=4):
	 ['K: original summary ratings meet standard', 'S: simple summary ratings meet standard', 'Y: added relevance ratings meet standard', 'AE: added relevance ratings meet standard']


In [47]:
index = 0
columns = choice_number_columns
classification=standards_columns[index]

plot_proportion_abs(df, columns, classification, label=1, height=200)

Top choice yes/no

In [48]:
index = 0
columns = choice_number_columns
classification = top_results_columns[index]
plot_proportion_abs(df, columns, classification, label=1, height=200)

# *End of Page*